# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [15]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# Creation of keySpace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project02 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
try:
    session.set_keyspace('project02')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
query = "CREATE TABLE IF NOT EXISTS eventLog"
query = query + "(sessionId text, itemInSession text, artist text, firstName text, gender text, lastName text, length text, level text, location text, song text, userId text, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
#     print("header: ", list(csvreader)[0])
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO eventLog (sessionId, itemInSession, artist, firstName, gender,  lastName, length, level, location, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[8], line[3], line[0], line[1], line[2], line[4], line[5], line[6], line[7], line[9], line[10]))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
query = "select song, artist, length from eventLog where sessionId = '338' and itemInSession = '4'"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)
t = PrettyTable(['Artist', 'Song', 'Length'])

for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [18]:
query = "CREATE TABLE IF NOT EXISTS userSession"
query = query + "(userId text, sessionId text, itemInSession text, artist text, firstName text, gender text, lastName text, length text, level text, location text, song text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [20]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO userSession (userId, sessionId, itemInSession, artist, firstName, gender, lastName, length, level, location, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10], line[8], line[3], line[0], line[1], line[2], line[4], line[5], line[6], line[7], line[9]))

In [25]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "select firstName, lastName, song, artist from userSession where userId = '10' and sessionId = '182'"
# query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int, PRIMARY KEY ((sessionId), itemInSession))"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)

    
t = PrettyTable(['First Name', 'Last Name', 'Song', 'Artist'])

for row in rows:
    t.add_row([row.firstname, row.lastname, row.song, row.artist])
print(t)

                    

+------------+-----------+------------------------------------------------------+-------------------+
| First Name | Last Name |                         Song                         |       Artist      |
+------------+-----------+------------------------------------------------------+-------------------+
|   Sylvie   |    Cruz   |                  Keep On Keepin' On                  |  Down To The Bone |
|   Sylvie   |    Cruz   |                     Greece 2000                      |    Three Drives   |
|   Sylvie   |    Cruz   |                      Kilometer                       | Sebastien Tellier |
|   Sylvie   |    Cruz   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Lonnie Gordon   |
+------------+-----------+------------------------------------------------------+-------------------+


In [26]:
query = "CREATE TABLE IF NOT EXISTS songHistory"
query = query + "(song text, firstName text, userId text, artist text, gender text, itemInSession text, lastName text, length text, level text, location text, sessionId text, PRIMARY KEY (song, firstName, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songHistory (song, firstName, userId, artist, gender, itemInSession, lastName, length, level, location, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[10], line[0], line[2], line[3], line[4], line[5], line[6], line[7], line[8]))

In [23]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "select firstName, lastName from songHistory where song='All Hands Against His Own'"
# query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int, PRIMARY KEY ((sessionId), itemInSession))"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)

t = PrettyTable(['First Name', 'Last Name'])

for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)

                    

Error from server: code=2200 [Invalid query] message="unconfigured table songhistory"
+------------+-----------+
| First Name | Last Name |
+------------+-----------+
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table eventLog"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table userSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table songHistory"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()